# 数理モデルからインスタンスを生成

JijModelingで数理モデルを記述した後、ソルバーで解くには、その数理モデルが持つPlaceholderにデータを代入する必要があります。この操作を実現するのが `Interpreter.eval_problem` メソッドです。このページでは、ナップサック問題を例に、OMMX Adapter経由でソルバーに解かせる方法を説明します。

ナップサック問題を定式化して得られる数理モデルは次の通りです：

$$
\begin{align*}
\mathrm{maximize} \quad & \sum_{i=0}^{N-1} v_i x_i \\
\mathrm{s.t.} \quad & \sum_{i=0}^{n-1} w_i x_i \leq W, \\
& x_{i} \in \{ 0, 1\} 
\end{align*}
$$

:::{note}
ナップサック問題の定式化について詳しく知りたい場合は [こちら](https://www.documentation.jijzept.com/docs/tutorial/knapsack/) を参照してください。
:::

この数理モデルにあるそれぞれの文字の意味は以下の通りです：

| 文字 | 説明 |
| --- | --- |
| $N$ |	アイテムの総数 |
| $v_{i}$ | アイテム $i$ の価値 |
| $w_{i}$ | アイテム $i$ の重さ |
| $W$ | ナップサックの耐荷重 |

この数理モデルの係数 $v_{i}, w_{i}, W$ に、次のインスタンスデータを入力して得られるインスタンスをソルバーで解くことを考えます：


| 文字 | 説明 |
| --- | --- |
| $\boldsymbol{v}$ | `[10, 13, 18, 31, 7, 15]` |
| $\boldsymbol{w}$ | `[11, 15, 20, 35, 10, 33]` |
| $W$ | `47` |

JijModelingを使うと、ソルバーに入力するためのインスタンスを次の3ステップで生成できます：

1. JijModelingでナップサック問題を定式化する
2. `Interpreter` オブジェクトにインスタンスデータを登録する
3. `Interpreter` オブジェクトを使って数理モデルをインスタンスに変換する

![Diagram of the process to generate an instance from a mathematical model](./assets/model_to_instance_01.png)

## Step1. JijModelingでナップサック問題を定式化する

JijModelingを使用してナップサック問題を定式化すると、以下のPythonコードになります：

In [1]:
import jijmodeling as jm

# アイテムの価値
v = jm.Placeholder("v", ndim=1)
# アイテムの重さ
w = jm.Placeholder("w", ndim=1)
# ナップサックの耐荷重
W = jm.Placeholder("W")
# アイテムの総数
N = v.len_at(0, latex="N")
# アイテムiをナップサックに入れる場合は1, 入れない場合は0を取る決定変数
x = jm.BinaryVar("x", shape=(N,)) 
# アイテムに割り当てられた番号を走る添え字
i = jm.Element("i", belong_to=(0, N))

problem = jm.Problem("problem", sense=jm.ProblemSense.MAXIMIZE)
# 目的関数
problem += jm.sum(i, v[i] * x[i])
# 制約条件: ナップサックの耐荷重を超えない
problem += jm.Constraint("重量制限", jm.sum(i, w[i] * x[i]) <= W)
problem

## Step2. Interpreter オブジェクトにインスタンスデータを登録する

Step1で定式化した数理モデルの係数に入力するデータ（インスタンスデータ）を用意します。これは `Interpreter` オブジェクトを使用して実現できます。

`Interpreter` クラスのコンストラクタの引数に、以下のペアを持つ辞書を渡すことでインスタンスデータを登録できます：

- キー：`Placeholder` オブジェクトの `name` プロパティに設定した文字列
- 値：入力するデータ

In [2]:
interpreter = jm.Interpreter({
    "v": [10, 13, 18, 31, 7, 15],  # アイテムの価値のデータ
    "w": [11, 15, 20, 35, 10, 33], # アイテムの重さのデータ
    "W": 47,                       # ナップサックの耐荷重のデータ
})

## Step3. Interpreter オブジェクトを使って数理モデルをインスタンスに変換する

数理モデルをインスタンスデータに変換するには、Interpreter.eval_problem メソッドを使用します。インスタンスデータが登録された Interpreter オブジェクトの eval_problem メソッドに Problem オブジェクトを渡すと、その Problem オブジェクトが持つ Placeholder にデータを入力し、インスタンスに変換して ommx.v1.Instance オブジェクトとして返します：

In [3]:
instance = interpreter.eval_problem(problem)

## OMMX Adapterに入力して最適化問題を解く

Step3で得られた `ommx.v1.Instance` オブジェクトをOMMX Adapterに入力することで、最適化計算を実行できます。
例えば、OMMX Adapterの一つである `ommx-python-mip-adapter` ([GitHub](https://github.com/Jij-Inc/ommx/tree/main/python/ommx-python-mip-adapter), [PyPI](https://pypi.org/project/ommx-python-mip-adapter/)) を使用して、そのインスタンスの最適解と最適値を求めるPythonコードは次の通りです：

In [4]:
import numpy as np
from ommx_python_mip_adapter import instance_to_model, model_to_solution

# ommx.v1.instanceオブジェクトからPython-MIPのModelオブジェクトに変換する
model = instance_to_model(instance)
# Python-MIPのデフォルトソルバー (SCIP) で最適化を実行する
model.optimize()
# Python-MIPによる最適化計算の結果をommx.v1.Solutionオブジェクトとして取得する
solution = model_to_solution(model, instance)

# 最適値は41である
assert np.isclose(solution.raw.objective, 41.0)
# x0 = x3 = 1, x1 = x2 = x4 = x5 = 0
assert solution.raw.state.entries == {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0}

補足として、 `ommx.v1.Solution` の `decision_variables` プロパティを使うことで、pandas.DataFrame オブジェクトとして最適解を取得・表示できます:

In [5]:
solution.decision_variables[["name", "subscripts", "value"]]

,name,subscripts,value
,,,
id,,,
0,x,[0],1.0
1,x,[1],0.0
2,x,[2],0.0
3,x,[3],1.0
4,x,[4],0.0
5,x,[5],0.0
